In [6]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.spatial.distance import cdist
from skimage.morphology import binary_erosion
import cv2
from scipy.spatial import cKDTree
from collections import defaultdict
from scipy import ndimage
from operator import add

In [7]:
path = r'C:\Users\Wojtek\Documents\Doktorat\Astral\data\Cont_AN_2_4'

abs_file = 'segmentation_absolute.h5'
dims_path = 'segmentation_dims.h5'

In [8]:
adf = pd.read_hdf(os.path.join(path, abs_file))
ddf = pd.read_hdf(os.path.join(path, dims_path))

In [9]:
adf.head(2)

,id,y,x,z,color
0,0,262,249,968,85
1,0,262,250,968,81


In [10]:
ddf.head(2)

,id,y_min,y_max,x_min,x_max,z_min,z_max,center_y,center_x,center_z
0,0,262,343,195,268,927,1035,302,231,981
1,1,161,272,592,716,692,724,216,654,708


In [11]:
ids = ddf.id.values

In [12]:
len(ids)

267

In [13]:
shape1 = adf.loc[adf['id'] == 2]
shape1 = shape1[['x', 'y', 'z']]

In [14]:
shape2 = adf.loc[adf['id'] == 5]

In [15]:
shape1 = shape1[['x', 'y', 'z']]
shape2 = shape2[['x', 'y', 'z']]

In [16]:
shape2.shape

(82969, 3)

In [17]:
shape1.shape

(117557, 3)

In [18]:
adf.shape

(2038618, 5)

In [19]:
a = [[0, 0, 0, 0, 0, 0],
[0, 0, 1, 1, 1, 0],
[0, 1, 1, 1, 1, 0],
[0, 1, 1, 1, 1, 0],
[0, 1, 1, 1, 0, 0],
[0, 0, 0, 0, 0, 0]]

In [20]:
shape1.x = shape1.x - shape1.x.min()
shape1.y = shape1.y - shape1.y.min()
shape1.z = shape1.z - shape1.z.min()

In [21]:
def get_border_inds(shape):
    indices = shape.values

    shape_np = np.ndarray(((indices[:, 0].max()+1, indices[:, 1].max()+1, indices[:, 2].max()+1)))

    for i in range(indices.shape[0]):
        shape_np[indices[i, 0], indices[i, 1], indices[i, 2]] = 1
        
    interior = binary_erosion(shape_np)
    
    border = np.logical_xor(shape_np, interior)
    
    border_inds = np.argwhere(border == True)
    
    return border_inds

In [22]:
shape1 = adf.loc[adf['id'] == 2]
shape2 = adf.loc[adf['id'] == 5]
shape1 = shape1[['x', 'y', 'z']]
shape2 = shape2[['x', 'y', 'z']]
border1 = get_border_inds(shape1)
border2 = get_border_inds(shape2)

In [23]:
shape1 = ddf.loc[ddf['id'] == 2]
tolerance_xy = 50
tolerance_z = 100

In [24]:
def get_tolerance_bounding_box(shape, tolerance_xy, tolerance_z):
    
    xl = shape['x_min'].values[0] - tolerance_xy
    yl = shape['y_min'].values[0] - tolerance_xy
    zl = shape['z_min'].values[0] - tolerance_z

    xu = shape['x_max'].values[0] + tolerance_xy
    yu = shape['y_max'].values[0] + tolerance_xy
    zu = shape['z_max'].values[0] + tolerance_z
    
    return (xl, xu), (yl, yu), (zl, zu)

In [25]:
xb, yb, zb = get_tolerance_bounding_box(shape1, tolerance_xy, tolerance_z)

In [26]:
def get_neighbor_shapes(shape, tolerance_xy, tolerance_z):
    
    xb, yb, zb = get_tolerance_bounding_box(shape, tolerance_xy, tolerance_z)
    
    neighbor_shapes = ddf.loc[( ddf['x_min'].between(*xb) ) | ( ddf['x_max'].between(*xb) )]
    neighbor_shapes = neighbor_shapes.loc[( neighbor_shapes['y_min'].between(*yb) ) | ( neighbor_shapes['y_max'].between(*yb) )]
    neighbor_shapes = neighbor_shapes.loc[( neighbor_shapes['z_min'].between(*zb) ) | ( neighbor_shapes['z_max'].between(*zb) )]
    
    return neighbor_shapes

In [27]:
def get_candidate_neighbors_dict(ddf, adf, tolerance_xy, tolerance_z):

    min_dist_dict = defaultdict(dict)

    ids = np.unique(ddf.id.values)

    for i in tqdm(ids):

        shape = ddf.loc[ddf['id'] == i]

        candidate_neighbors = get_neighbor_shapes(shape, tolerance_xy, tolerance_z)

        candidate_ids = np.unique(candidate_neighbors.id.values)

        for j in candidate_ids:
            if i != j:
                shape1 = adf.loc[adf['id'] == i]
                shape2 = adf.loc[adf['id'] == j]
                shape1 = shape1[['x', 'y', 'z']]
                shape2 = shape2[['x', 'y', 'z']]
                #border1 = get_border_inds(shape1)
                #border2 = get_border_inds(shape2)
                
                shape1_xy = shape1[['x', 'y']]
                shape2_xy = shape2[['x', 'y']]
                
                shape1_z = np.unique(shape1[['z']].values)
                shape2_z = np.unique(shape2[['z']].values)
                
                shape1_z = np.expand_dims(shape1_z, 1)
                shape2_z = np.expand_dims(shape2_z, 1)
                
                min_dists_xy, min_dist_idx_xy = cKDTree(shape1_xy).query(shape2_xy, 1)
                min_dists_z, min_dist_idx_z = cKDTree(shape1_z).query(shape2_z, 1)
    #            min_dists, min_dist_idx = cKDTree(border1).query(border2, 1)
                #min_dists = (min_dists_xy.min(), min_dists_z.min())

                min_dist_dict[i][j] = ( min_dists_xy.min(), min_dists_z.min() )
                min_dist_dict[j][i] = ( min_dists_xy.min(), min_dists_z.min() )
    return min_dist_dict


    #             mat = cdist(border1, border2)

    #             min_dist = mat.min()

    #             min_dist_dict[i][j] = min_dist
    #             min_dist_dict[j][i] = min_dist

In [28]:
dist_dict = get_candidate_neighbors_dict(ddf, adf, tolerance_xy, tolerance_z)

100%|████████████████████████████████████████████████████████████████████████████████| 267/267 [00:42<00:00,  6.28it/s]


In [29]:
dist_dict

defaultdict(dict,
            {0: {3: (12.041594578792296, 24.0),
              9: (7.615773105863909, 0.0),
              11: (21.37755832643195, 0.0),
              17: (0.0, 57.0),
              32: (35.11409973215888, 0.0),
              36: (31.953090617340916, 75.0),
              37: (67.67569726275453, 29.0),
              49: (41.7612260356422, 2.0),
              60: (0.0, 86.0),
              61: (12.806248474865697, 0.0),
              67: (0.0, 28.0),
              72: (0.0, 0.0),
              119: (35.4682957019364, 0.0),
              121: (46.861498055439924, 94.0),
              135: (27.892651361962706, 0.0),
              147: (0.0, 2.0),
              169: (8.94427190999916, 54.0),
              173: (34.0, 94.0),
              200: (12.529964086141668, 0.0),
              207: (0.0, 45.0),
              243: (0.0, 8.0),
              255: (60.108235708594876, 24.0)},
             3: {0: (12.041594578792296, 24.0),
              9: (0.0, 99.0),
              11: (4

In [ ]:
# postprocess

def filter_distant_neighbors(dist_dict, tolerance_xy, tolerance_z):
    for shape_id in dist_dict.keys():
        candidate_neighbor_dict = dist_dict[shape_id]
        filtered_candidate_dict = dict(filter(lambda y: y[1][0] < tolerance_xy and y[1][1] < tolerance_z, candidate_neighbor_dict.items()))
        dist_dict[shape_id] = filtered_candidate_dict
    return dist_dict

In [32]:
dist_dict = filter_distant_neighbors(dist_dict, tolerance_xy, tolerance_z)

In [34]:
def calculate_euc_dists(ddf, shape1, shape2):
    xy_axis1 = ddf.loc[ddf.id == shape1, ['center_y', 'center_x']].values
    xy_axis2 = ddf.loc[ddf.id == shape2, ['center_y', 'center_x']].values
    center_dist_xy = cdist(xy_axis1, xy_axis2)[0][0]

    z_axis1 = ddf.loc[ddf.id == shape1, ['center_z']].values
    z_axis1 = np.vstack([z_axis1, np.zeros((z_axis1.shape[0]))]).T
    z_axis2 = ddf.loc[ddf.id == shape2, ['center_z']].values
    z_axis2 = np.vstack([z_axis2, np.zeros((z_axis2.shape[0]))]).T

    center_dist_z = cdist(z_axis1, z_axis2)[0][0]
    
    return center_dist_xy, center_dist_z

In [35]:
def calculate_com_dists(ddf, adf, shape1_id, shape2_id):

    shape1 = adf.loc[adf['id'] == shape1_id]
    shape2 = adf.loc[adf['id'] == shape2_id]
    shapes = [shape1, shape2]

    shapes = list(map(lambda df: df[['x', 'y', 'z']], shapes))

    offsets = []

    coms = []

    for shape in shapes: 
        offsets.append([shape.x.min(), shape.y.min(), shape.z.min()])
        shape.x = shape.x - shape.x.min()
        shape.y = shape.y - shape.y.min()
        shape.z = shape.z - shape.z.min()

        indices = shape.values

        shape_np = np.zeros(((indices[:, 0].max()+1, indices[:, 1].max()+1, indices[:, 2].max()+1)))

        for i in range(indices.shape[0]):
            shape_np[indices[i, 0], indices[i, 1], indices[i, 2]] = 1
        
        #print(np.unique(shape_np))
        com = ndimage.measurements.center_of_mass(shape_np)
        #print(com)
        com = list(map(lambda x: int(x), com))
        coms.append(com)

    coms_offset = []

    for com, offset in zip(coms, offsets):
        coms_offset.append( list( map(add, com, offset) ) )

    xy1, xy2 = coms_offset[0][:2], coms_offset[1][:2]
    xy1, xy2 = np.array(xy1), np.array(xy2)
    xy1, xy2 = np.expand_dims(xy1, -1).T, np.expand_dims(xy2, -1).T
    z1, z2 = coms_offset[0][2], coms_offset[1][2]

    com_dist_z = abs(z1-z2)
    com_dist_xy = cdist(xy1, xy2)[0][0]

    return com_dist_xy, com_dist_z

In [36]:
dist_df = pd.DataFrame(columns=['shape_id_1', 'shape_id_2', 'center_dist_xy', 'center_dist_z', 'center_of_mass_dist_xy', 'center_of_mass_dist_z'])

for shape_id, neighbor_dict in tqdm(dist_dict.items()):
    shape1 = shape_id
    for shape2 in neighbor_dict.keys():
        center_dist_xy, center_dist_z = calculate_euc_dists(ddf, shape1, shape2)
        
        com_dist_xy, com_dist_z = calculate_com_dists(ddf, adf, shape1, shape2)
        
        
        row = {
            'shape_id_1' : shape1,
            'shape_id_2' : shape2,
            'center_dist_xy' : center_dist_xy,
            'center_dist_z' : center_dist_z,
            'center_of_mass_dist_xy' : com_dist_xy,
            'center_of_mass_dist_z' : com_dist_z
        }
        
        dist_df = dist_df.append(row, ignore_index=True)
dist_df = dist_df.astype('int')

  0%|                                                                                          | 0/245 [00:00<?, ?it/s]c:\programdata\anaconda3\envs\astro\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
100%|████████████████████████████████████████████████████████████████████████████████| 245/245 [00:45<00:00,  5.44it/s]


In [37]:
dist_df

,shape_id_1,shape_id_2,center_dist_xy,center_dist_z,center_of_mass_dist_xy,center_of_mass_dist_z
0,0,3,82,104,79,108
1,0,9,80,32,76,30
2,0,11,104,29,97,32
3,0,17,10,124,4,122
4,0,32,102,38,99,37
...,...,...,...,...,...,...
1239,218,138,57,8,58,9
1240,137,168,10,76,10,77
1241,168,137,10,76,10,77
1242,244,260,1,94,1,94


In [42]:
dist_df = dist_df.sort_values(by=['shape_id_1'])

In [44]:
dist_df.to_csv(r'C:\Users\Wojtek\Documents\Doktorat\Astral\data\Cont_AN_2_4\neighbors.csv', index=False)

In [232]:
dist_df

,shape_id_1,shape_id_2,center_dist_xy,center_dist_z,center_of_mass_dist_xy,center_of_mass_dist_z
0,0,3,82,104,79,108
1,0,5,166,45,169,46
2,0,9,80,32,76,30
3,0,11,104,29,97,32
4,0,17,10,124,4,122
...,...,...,...,...,...,...
1923,218,154,79,60,81,60
1924,218,161,66,52,66,51
1925,218,176,74,5,75,5
1926,244,260,1,94,1,94


In [209]:
xy1.T

array([[659],
       [ 56]])

In [210]:
xy2.T

array([[ 68],
       [280]])

In [211]:
com_dist_xy

632.0261070557133

In [176]:
from operator import add

In [ ]:

indices = shape.values

In [144]:
indices = shape.values

shape_np = np.ndarray(((indices[:, 0].max()+1, indices[:, 1].max()+1, indices[:, 2].max()+1)))

for i in range(indices.shape[0]):
    shape_np[indices[i, 0], indices[i, 1], indices[i, 2]] = 1

In [154]:


com = ndimage.measurements.center_of_mass(shape_np)
com = tuple(map(lambda x: int(x), com))

In [155]:
com

(266, 568, 575)

In [143]:
dist_df

,shape_id_1,shape_id_2,center_dist_xy,center_dist_z
0,0,3,418,104
1,0,5,645,45
2,0,9,420,32
3,0,11,545,29
4,0,17,490,124
...,...,...,...,...
1923,218,154,374,60
1924,218,161,422,52
1925,218,176,389,5
1926,244,260,288,94


In [122]:
row = {
            'shape_id_1' : [shape1],
            'shape_id_2' : [shape2],
            'center_dist_xy' : [center_dist_xy],
            'center_dist_z' : [center_dist_z],
        }

In [123]:
row

{'shape_id_1': [110],
 'shape_id_2': [242],
 'center_dist_xy': [285.70089254323307],
 'center_dist_z': [10.0]}

In [125]:
a = pd.DataFrame.from_dict(row)

,shape_id_1,shape_id_2,center_dist_xy,center_dist_z
0,110,242,285.700893,10.0
1,110,242,285.700893,10.0


In [112]:
xy_axis = ddf.loc[ddf.id == 2, ['center_y', 'center_x']].values

In [113]:
xy_axis2 = ddf.loc[ddf.id == 5, ['center_y', 'center_x']].values

In [114]:
cdist(xy_axis, xy_axis2)[0][0]

645.2480143324735

In [101]:
shape1 = adf.loc[adf['id'] == 10]
shape2 = adf.loc[adf['id'] == 11]
shape1 = shape1[['x', 'y', 'z']]
shape2 = shape2[['x', 'y', 'z']]
border1 = get_border_inds(shape1)
border2 = get_border_inds(shape2)

mat = cdist(border1, border2)

min_dist = mat.min()

KeyboardInterrupt: 

In [97]:
min_dist

288.91867367825154

In [98]:
from scipy.spatial import cKDTree
min_dists, min_dist_idx = cKDTree(border1).query(border2, 1)

In [99]:
min_dists.min()

288.91867367825154

In [59]:
indices = shape1.values

In [67]:
shape_np = np.ndarray(((indices.max()+1, indices.max()+1, indices.max()+1)))

for i in range(indices.shape[0]):
    shape_np[indices[i, 0], indices[i, 1], indices[i, 2]] = 1

In [68]:
np.unique(shape_np)

array([0., 1.])

In [148]:
border1 = get_border_inds(shape1)
border2 = get_border_inds(shape2)

In [149]:
border2

array([[ 11,   0,   0],
       [ 11,   0,   1],
       [ 11,   0,   2],
       ...,
       [924, 949, 947],
       [924, 949, 948],
       [924, 949, 949]], dtype=int64)

In [103]:
indices = shape1.values

for i in range(indices.shape[0]):
    s1[indices[i, :]] = 1

In [106]:
b = binary_erosion(s1)

In [143]:
border = np.logical_xor(s1, b)

In [144]:
border_inds = np.argwhere(border == True)

In [145]:
border_inds.shape

(15129, 3)

In [111]:
s1.shape

(123, 123, 123)

In [112]:
b.shape

(123, 123, 123)

In [105]:
s1.shape

(123, 123, 123)

In [88]:
shape1.values.shape

(117557, 3)

In [87]:
shape1.z.max()

55

In [84]:
s1.shape

(113, 123, 57)

In [67]:
a = np.array(a) 

In [64]:
b = binary_erosion(a)

In [97]:
xy_axis = ids[['center_y', 'center_x']]

In [98]:
xy_dist = cdist(xy_axis, xy_axis)

In [99]:
z_axis = np.vstack([z_axis, np.zeros((z_axis.shape[0]))]).T

In [100]:
z_dist = cdist(z_axis, z_axis)

In [70]:
xy_dist

array([[  0.        ,  20.51828453,  20.51828453,  65.60487787,
         19.20937271, 118.13974776],
       [ 20.51828453,   0.        ,   0.        ,  86.05230967,
          2.        , 128.99612397],
       [ 20.51828453,   0.        ,   0.        ,  86.05230967,
          2.        , 128.99612397],
       [ 65.60487787,  86.05230967,  86.05230967,   0.        ,
         84.81155582, 111.19802157],
       [ 19.20937271,   2.        ,   2.        ,  84.81155582,
          0.        , 127.01181047],
       [118.13974776, 128.99612397, 128.99612397, 111.19802157,
        127.01181047,   0.        ]])

In [32]:
ids = ddf.loc[ddf.center_x.between(lx, ux)]

In [35]:
ids.center_y.max()

576

In [26]:
ids

,id,y_min,y_max,x_min,x_max,z_min,z_max,center_y,center_x,center_z
0,0,262,343,195,268,927,1035,302,231,981
3,3,211,303,257,344,852,903,257,300,877
9,9,210,298,252,340,1002,1025,254,296,1013
10,10,258,347,194,268,1159,1197,302,231,1178
11,11,204,264,96,208,929,976,234,152,952
...,...,...,...,...,...,...,...,...,...,...
256,256,195,207,360,375,803,806,201,367,804
257,257,409,421,161,175,1122,1123,415,168,1122
262,262,171,177,330,349,33,37,174,339,35
264,264,387,396,257,264,281,284,391,260,282


In [17]:
df.center_x.max()

955

In [18]:
df.center_y.max()

602